In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

In [5]:
#Problem 1
#A farmer owns 1000 acres of land. Their options for using the land are cattle, plant wheat, plant corn or plant tomatoes
#Cattle requires 4 acres of land for one head of cattle. Anually, 12,000 hours of labor are availble
#it is required that at least 20% of the farmland that is cultivated must be used for cattle breeding, at most 30% of the available farmland can be used for growing tomatoes, and the ratio
#between the amount of farmland assigned to growing wheat and that is left uncultivated should not exceed 2 to 1
#                           Cattle     Wheat       Corn        Tomatoes
#                   Profit  1600/head   5/bushel    6/bushel    0.5/lb
#           yield per acre  0.25/ head  50 bushel   80 bushel   1000 lb
#annual labor requirement   40 hr/head  10 hr/acre  12 hr/acre  25 hr/acre



#define equations for the constraints of profit, yild per acre, labor requirements, cultivated and uncultivated land use
def farmprofit(x):
    cattle,wheat,corn,tomatoes = x
    #units should be in terms of acres, each variable the number of acres for that product
    return -(1600*0.25*cattle + 5*50*wheat + 6*80*corn + 0.5*1000*tomatoes) 
#negative because we want to maximize profit

def acre(x):
    #inequality: cultivated land <= 1000
    cattle,wheat,corn,tomatoes = x
    return 1000 - (cattle + wheat + corn + tomatoes) 

def labor(x):
    #inequality: labor <= 12000
    cattle,wheat,corn,tomatoes = x
    return 12000 - (40*0.25*cattle + 10*wheat + 12*corn + 25*tomatoes)

def wheat_ratio(x):
    #inequality: ratio of wheat land/unusedland <= 2
    cattle,wheat,corn,tomatoes = x
    uncultivated = acre(x)
    return 2 - (wheat/uncultivated)

def cattle_acre(x):
    #inequality: cattle breeding land >= 0.20*culitvated land
    cattle,wheat,corn,tomatoes = x
    cultivated = acre(x) + 1000 #add 1000 since acre(x) output subtracts 1000
    return cattle - 0.20*cultivated

#create constraint so you cannot have fraction of cattle
#takes 4 acres to support 1 head of cattle
def cattlehead(x):
    cattle,wheat,corn,tomatoes = x
    return cattle % 4
cons = ({'type': 'ineq', 'fun': acre},
       {'type': 'ineq','fun': labor},
       {'type':'ineq','fun':wheat_ratio},
       {'type':'ineq','fun':cattle_acre},
       {'type':'eq','fun':cattlehead})

#bounds determined by the ammount of land available 
cattle_bnds = [0,1000] #1000 acres * 0.25 head/acre
wheat_bnds = [0,1000] #1000 acres * 50 bushels/acre
corn_bnds = [0,1000] #1000 acres * 80 bushels/acre
#tomatoe can use up to 30% of the total land available
tomatoe_bnds = [0,0.3*1000] #1000 acres * 1000 lbs/acre

bnds = [cattle_bnds,wheat_bnds,corn_bnds,tomatoe_bnds]

guess = [200,200,200,200]
sol = minimize(farmprofit, guess, bounds = bnds, constraints = cons)

cattle,wheat,corn,tomatoes = sol.x

print('Profit: $%.2f'% -farmprofit(sol.x))
print('Cattle Acres: %.2f acres'%cattle)
print('Wheat Acres: %.2f acres'%wheat)
print('Corn Acres: %.2f acres'%corn)
print('Tomatoes Acres: %.2f acres'%tomatoes)
print()
print('Total Cattle: %.2f head'%(0.25*cattle))
print('Total Wheat: %.2f bushels'%(50*wheat))
print('Total Corn: %.2f bushels'%(80*corn))
print('Total Tomatoes: %.2f lbs'%(1000*tomatoes))
print()
print('Cultivated land: %.2f acres' %(cattle+wheat+tomatoes+corn))
print('Uncultivated land: %.2f acres'%acre(sol.x))
print('Total Labor: %.2f hours' %(labor(sol.x)+12000))

sol.success

Profit: $464615.38
Cattle Acres: 200.00 acres
Wheat Acres: 0.00 acres
Corn Acres: 769.23 acres
Tomatoes Acres: 30.77 acres

Total Cattle: 50.00 head
Total Wheat: 0.00 bushels
Total Corn: 61538.46 bushels
Total Tomatoes: 30769.23 lbs

Cultivated land: 1000.00 acres
Uncultivated land: 0.00 acres
Total Labor: 12000.00 hours


True

In [7]:
#Problem 2
/// Have an inventory of 120 20ft rods, 160 15ft rods, and 40 8ft rods. Your customer requires 200 10ft rods, and 250 6ft rods. 
some rods will need to be cut in order to satisfy the customer requirements
Using optimization, determien the solution that minimizes waste
///

#def equations so that the total lengths of 10 ft rods and 6 ft rods add up to what they should be 
def r10length(x):
    #equality: total length of 10ft rods should be 10ft * 200 rods
    r20a,r15a,r8a,r20b,r15b,r8b = x
    #any 20 ft rods used here will yield 10 ft waste, and 15 ft rods yield 5 ft waste
    r10length = 10*200
    return r10length - (2*10*r20a) - (10*r15a)

def r6length(x): 
   #equality: total length of 6ft rods should be 6ft * 250 rods
    r20a,r15a,r8a,r20b,r15b,r8b = x
    r6length = 6*250
    #mulitiply by 20 ft rod by 3 and 6 because 3 6ft rods can be produced, can produce 2 6ft rods with 15 ft rod
    return r6length - (3*6*r20b) - (2*6*r15b) - (6*r8b)

#def waste as the number of 20 ft, 15 ft, and 8 ft rods that were cut to 10 ft and 6 ft
def waste(x):
    #waste: r20a 10ft, r15a 5ft, r8a 0ft, r20b 14 ft, r15b 9 ft, r8b 2 ft
    r20a,r15a,r8a,r20b,r15b,r8b = x
    #0 ft wasted when cutting 20 ft rod into 10 ft, 5 ft wasted when cutting 15 ft rod to 10 ft, etc
    wasted = (0*r20a) + (5*r15a) + (0*r8a) + (2*r20b) + (3*r15b) + (2*r8b)
    return wasted - 0

#define equations so that the number of 20 foot rods, 15 ft rods, and 8 ft rods cut to 10 adn 6 ft add up to the 
#total number of rods in stock 
def r20tot(x):
    #inequality: total number 20ft rods <= stock 
    #120 20ft rods in stock 
    r20a,r15a,r8a,r20b,r15b,r8b = x
    return 120 - r20a - r20b

def r15tot(x):
    #inequality: total number 15ft rods <= stock  
    #160 15ft rods in stock 
    r20a,r15a,r8a,r20b,r15b,r8b = x
    return 160 - r15a - r15b

def r8tot(x):
    #inequality: total number 8ft rods <= stock 
    #40 8ft rods in stock 
    r20a,r15a,r8a,r20b,r15b,r8b = x
    return 40 - r8a - r8b

cons = ({'type': 'eq', 'fun': r6length},
       {'type': 'eq','fun': r10length},
       {'type': 'ineq','fun': r20tot},
       {'type': 'ineq','fun': r15tot},
       {'type': 'ineq','fun': r8tot})


bnd = []
for i in range(6):
    bnd.append([0,None])
    
guess = [60,80,20,60,80,20]

sol = minimize(waste,guess,bounds = bnd, constraints = cons)

r20a,r15a,r8a,r20b,r15b,r8b = sol.x

print('Number of 20ft rods cut to 10 ft: %i' %r20a)
print('Number of 15ft rods cut to 10 ft: %i' %r15a)
print('Number of 8ft rods cut to 10 ft: %i' %r8a)
print('Number of 20ft rods cut to 6 ft: %i' %r20b)
print('Number of 15ft rods cut to 6 ft: %i' %r15b)
print('Number of 8ft rods cut to 6 ft: %i' %r8b)
print()
print('Total wasted length: %i ft' %waste(sol.x))
print('Rod 20ft waste: %i ft' %(2*r20b))
print('Rod 15ft waste: %i ft' %(5*r15a + 3*r15b))
print('Rod 8ft waste: %i ft' %(2*r8b))
sol.success

Number of 20ft rods cut to 10 ft: 99
Number of 15ft rods cut to 10 ft: 0
Number of 8ft rods cut to 10 ft: 20
Number of 20ft rods cut to 6 ft: 20
Number of 15ft rods cut to 6 ft: 94
Number of 8ft rods cut to 6 ft: 0

Total wasted length: 325 ft
Rod 20ft waste: 40 ft
Rod 15ft waste: 285 ft
Rod 8ft waste: 0 ft


True

In [9]:
#Problem 3 
/// A refiner produces two grades of gasoline by blending four different stocks. Goal is to maximize profit
                    Raw feed stock
        stock      availability        price
        1           3000                13
        2           2000                15.30
        3           4000                14.60
        4           1000                14.90

                    Product requirements
        grade       specification               price
        A           Not more than 15% of 1      16.20
                    Not less than 40% of 2
                    Not more than 50% of 3

        B           Not more than 10% of 1      15.75
                    Not less than 10% of 2
///


#units are in gallons
#up front cost of buying stock to make the blends for profit
def gasprofit(x):
    s1a,s1b,s2a,s2b,s3a,s3b,s4a,s4b = x
    cost = 13*(s1a+s1b) + 15.3*(s2a+s2b) + 14.6*(s3a+s3b) + 14.9*(s4a + s4b)
    aprofit = 16.2*gradeA(x)
    bprofit = 15.75*gradeB(x)
    profit = aprofit + bprofit - cost
    #maximize by making negative 
    return -profit

#create functions that limit the amount of stock that can be used
def stock1tot(x):
    s1a,s1b,s2a,s2b,s3a,s3b,s4a,s4b = x
    return 3000 - (s1a+s1b)

def stock2tot(x):
    s1a,s1b,s2a,s2b,s3a,s3b,s4a,s4b = x
    return 2000 - (s2a+s2b)

def stock3tot(x):
    s1a,s1b,s2a,s2b,s3a,s3b,s4a,s4b = x
    return 4000 - (s3a+s3b)

def stock4tot(x):
    s1a,s1b,s2a,s2b,s3a,s3b,s4a,s4b = x
    return 1000 - (s4a+s4b)

#create restrictions for blend A
def gradeAstock1(x):
    s1a,s1b,s2a,s2b,s3a,s3b,s4a,s4b = x
    return 0.15*gradeA(x) - s1a

def gradeAstock2(x):
    s1a,s1b,s2a,s2b,s3a,s3b,s4a,s4b = x
    return s2a - 0.40*gradeA(x)

def gradeAstock3(x):
    s1a,s1b,s2a,s2b,s3a,s3b,s4a,s4b = x
    return 0.50*gradeA(x) - s3a

#create restrictions for blend B
def gradeBstock1(x):
    s1a,s1b,s2a,s2b,s3a,s3b,s4a,s4b = x
    return 0.10*gradeB(x) - s1b

def gradeBstock2(x):
    s1a,s1b,s2a,s2b,s3a,s3b,s4a,s4b = x
    return s2b - 0.10*gradeB(x)

#upper bounds are created in the above functions, so None is used as the upper bound
stock1abnd = [0,None]
stock1bbnd = [0,None]
stock2abnd = [0,None]
stock2bbnd = [0,None]
stock3abnd = [0,None]
stock3bbnd = [0,None]
stock4abnd = [0,None]
stock4bbnd = [0,None]

bnds = [stock1abnd,stock1bbnd,stock2abnd,stock2bbnd,stock3abnd,stock3bbnd,stock4abnd,stock4bbnd]

cons = ({'type': 'ineq','fun': stock1tot},
       {'type': 'ineq','fun': stock2tot},
       {'type': 'ineq','fun': stock3tot},
       {'type': 'ineq','fun': stock4tot},
       {'type': 'ineq','fun': gradeAstock1},
       {'type': 'ineq','fun': gradeAstock2},
       {'type': 'ineq','fun': gradeAstock3},
       {'type': 'ineq','fun': gradeBstock1},
       {'type': 'ineq','fun': gradeBstock2})

guess1a = 1000
guess1b = 1000
guess2a = 1000
guess2b = 1000
guess3a = 1000
guess3b = 1000
guess4a = 1000
guess4b = 1000

guess = [guess1a,guess1b, guess2a, guess2b, guess3a, guess3b, guess4a, guess4b]
sol = minimize(gasprofit, guess, bounds = bnds, constraints = cons)

s1a,s1b,s2a,s2b,s3a,s3b,s4a,s4b = sol.x

print( 'Profit: $%.2f'%(-gasprofit(sol.x)) )
print()
print( 'Blend A: %.2f gal' %(gradeA(sol.x)) ) 
print( 'Stock 1 used in blend A: %.2f gal' %s1a )
print( 'Stock 2 used in blend A: %.2f gal' %s2a )
print( 'Stock 3 used in blend A: %.2f gal' %s3a )
print()
print( 'Blend B: %.2f gal' %(gradeB(sol.x)) ) 
print( 'Stock 1 used in blend B: %.2f gal' %s1b )
print( 'Stock 2 used in blend B: %.2f gal' %s2b )
print()
print( 'Stock 1 leftover: %.2f gal' %(stock1tot(sol.x)) )
print( 'Stock 2 leftover: %.2f gal' %(stock2tot(sol.x)) )
print( 'Stock 3 leftover: %.2f gal' %(stock3tot(sol.x)) )
print( 'Stock 4 leftover: %.2f gal' %(stock4tot(sol.x)) )
sol.success

Profit: $10900.00

Blend A: 4000.00 gal
Stock 1 used in blend A: 600.00 gal
Stock 2 used in blend A: 1600.00 gal
Stock 3 used in blend A: 1650.00 gal

Blend B: 4000.00 gal
Stock 1 used in blend B: 400.00 gal
Stock 2 used in blend B: 400.00 gal

Stock 1 leftover: 2000.00 gal
Stock 2 leftover: -0.00 gal
Stock 3 leftover: -0.00 gal
Stock 4 leftover: -0.00 gal


True